In [1]:
import cv2
import numpy as np

import torch
import torchvision.transforms as transforms
# from PIL import Image, ImageDraw
#import PIL
from PIL import Image as PILImage
from PIL import ImageDraw as PILImageDraw

import matplotlib.pyplot as plt
import argparse
import os
from IPython.display import clear_output, display 
from IPython.display import Image as IPImage
from facenet_pytorch import MTCNN, InceptionResnetV1

from model import *


In [2]:
cuda_device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'Running on device: {cuda_device}')

Running on device: cuda:0


In [3]:
face_detect_model = MTCNN(keep_all=True, device=cuda_device)

In [4]:
#Emotion recognition
def load_trained_model(model_path):
    model = Face_Emotion_CNN()
    model.load_state_dict(torch.load(model_path, map_location=lambda storage, loc: storage), strict=False)
    return model


model = load_trained_model('../models/emotion_m2pt.pt')
emotion_dict = {0: 'Neutral', 1: 'Happy', 2: 'Surprise', 3: 'Sad',
                    4: 'Angry', 5: 'Disgust', 6: 'Fear'}


val_transform = transforms.Compose([
        transforms.ToTensor()])

In [7]:
video_capture = cv2.VideoCapture(0)
frame_count = 0


def get_name(face_img):
    return 'josh'

def get_emotion(face_img):
    return 'happy'

from PIL import Image

try:
    while(True):
        frame_count += 1
        
        clear_output(wait=True)
        _, frame = video_capture.read()
        faces, _ = face_detect_model.detect(frame)
        
        if faces is not None:
            
            
            for top, left, bottom, right  in faces:
                
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
                resize_frame = cv2.resize(gray[int(top):int(bottom), int(left):int(right)], (48, 48))

                X = resize_frame/256
                X = PILImage.fromarray((X))
                X = val_transform(X).unsqueeze(0)
                with torch.no_grad():
                    model.eval()
                    log_ps = model.cpu()(X)
                    ps = torch.exp(log_ps)
                    top_p, top_class = ps.topk(1, dim=1)
                    pred = emotion_dict[int(top_class.numpy())] 
        
                face_img = frame[0:300, 0:200]
                cv2.rectangle(frame, (top, left), (bottom, right), (0, 100, 0), 2)


                font = cv2.FONT_HERSHEY_DUPLEX
                x, y, w, h = int(top+2), int(right-50), int(left-2), int(bottom-26)

                scale = 0.045 # this value can be from 0 to 1 (0,1] to change the size of the text relative to the image
                fontScale = min(w,h)/(25/scale)

                cv2.putText(frame, "Name", ((int(top)+3), (int(right)-18)), font, fontScale, (255, 255, 255), 1, cv2.LINE_AA)
                cv2.putText(frame, pred, ((int(top)+3), (int(right)-4)), font, fontScale, (255, 255, 255), 1, cv2.LINE_AA)
               
   
        ret, img = cv2.imencode('.jpg', frame)
        img = IPImage(data=img)
        display(img)
        
        # Display the resulting image
        #cv2.imshow('Video', frame)

        # Hit 'q' on the keyboard to quit!
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break
finally:
    video_capture.release()
  
        
    

error: /build/opencv-XDqSFW/opencv-3.2.0+dfsg/modules/imgproc/src/imgwarp.cpp:3492: error: (-215) ssize.width > 0 && ssize.height > 0 in function resize
